# L4: Tool Calling

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Import libraries

In [ ]:
from ai21 import AI21Client
from ai21.logger import set_verbose
from ai21.models.chat import ChatMessage, ToolMessage, FunctionToolDefinition, ToolDefinition, ToolParameters
import json, requests, re
from bs4 import BeautifulSoup

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>utils.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

## Load API key and create AI21Client

In [ ]:
from utils import get_ai21_api_key
ai21_api_key = get_ai21_api_key()

client = AI21Client(api_key=ai21_api_key)

# Arithmetic tools for Jamba

In [ ]:
def multiplication(a: float, b: float) -> float:
    print(f"Multiplying {a} and {b}...")
    return a * b

def addition(a: float, b: float) -> float:
    print(f"Adding {a} and {b}...")
    return a + b

In [ ]:
multiplication_tool = ToolDefinition(
    type="function",
    function=FunctionToolDefinition(
        name="multiplication",
        description="Multiply two numbers",
        parameters=ToolParameters(
            type="object",
            properties={
                "a": {"type": "number", "description": "The first number to multiply"},
                "b": {"type": "number", "description": "The second number to multiply"}
            },
            required=["a", "b"],
        ),
    ),
)

In [ ]:
addition_tool = ToolDefinition(
    type="function",
    function=FunctionToolDefinition(
        name="addition",
        description="Add two numbers",
        parameters=ToolParameters(
            type="object",
            properties={
                "a": {"type": "number", "description": "The first number to add"},
                "b": {"type": "number", "description": "The second number to add"}
            },
            required=["a", "b"],
        ),
    ),
)

In [ ]:
tools = [multiplication_tool, addition_tool]

In [ ]:
messages = [
    ChatMessage(role="system", content="You are a helpful math assistant. Use the supplied tools to assist the user."),
    ChatMessage(role="user", content="What is the capital of France")
]

In [ ]:
response = client.chat.completions.create(
    messages=messages, 
    model="jamba-1.5-large", 
    tools=tools)
response.choices[0].message

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

In [ ]:
messages = [
    ChatMessage(role="system", content="You are a helpful math assistant. Use the supplied tools to assist the user."),
    ChatMessage(role="user", content="Can you help me multiply 62.74 and 3.5?")
]

In [ ]:
response = client.chat.completions.create(messages=messages, model="jamba-1.5-large", tools=tools)
response.choices[0].message

In [ ]:
assistant_message = response.choices[0].message
tool_call = assistant_message.tool_calls[0]
tools_parameters_dict = json.loads(tool_call.function.arguments)

tool_result = multiplication(tools_parameters_dict["a"], tools_parameters_dict["b"])
tool_result

In [ ]:
tool_message = ToolMessage(role="tool", 
                           tool_call_id=tool_call.id, 
                           content=str(tool_result))

In [ ]:
messages.append(assistant_message)
messages.append(tool_message)
response_final = client.chat.completions.create(
    messages=messages, 
    model="jamba-1.5-large")
response_final.choices[0].message.content

In [ ]:
tools_function = {"multiplication": multiplication, "addition": addition}
function_name = tool_call.function.name
tools_parameters = list(json.loads(tool_call.function.arguments).values())
tool_result = tools_function[function_name](*tools_parameters)
tool_result

## SEC 10-Q analysis tool calling with Jamba

In [ ]:
from utils import sec_10q

In [ ]:
tool_definition = ToolDefinition(
    type="function",
    function=FunctionToolDefinition(
        name="sec_10q",
        description="Retrieve the full text of the latest 10-Q filing for a given company ticker",
        parameters=ToolParameters(
            type="object",
            properties={
                "ticker": {"type": "string", "description": "The ticker symbol of the company"}
            },
            required=["ticker"],
        ),
    ),
)

In [ ]:
tools = [tool_definition]

In [ ]:
messages = [
    ChatMessage(
        role="system",
        content="You are a helpful financial assistant. Use the supplied tools to assist the user with quarterly financial data.",
    ),
    ChatMessage(
        role="user", 
        content="Provide a summary of Nvidia's most recent quarterly 10-Q filing"),
]

In [ ]:
response = client.chat.completions.create(
    messages=messages, 
    model="jamba-1.5-large", 
    tools=tools)
response

In [ ]:
assistant_message = response.choices[0].message
tool_call = assistant_message.tool_calls[0]
tools_parameters_dict = json.loads(tool_call.function.arguments)

tool_result = sec_10q(tools_parameters_dict["ticker"])
tool_result

In [ ]:
messages.append(assistant_message)

tool_message = ToolMessage(role="tool", 
                           tool_call_id=tool_call.id, 
                           content=json.dumps(tool_result))
messages.append(tool_message)

response_final = client.chat.completions.create(
    messages=messages, 
    model="jamba-1.5-large")
print(response_final.choices[0].message.content)

In [ ]:
messages = [
    ChatMessage(
        role="system",
        content="You are a helpful financial assistant. Use the supplied tools to assist the user with quarterly financial data.",
    ),
    ChatMessage(role="user", content="Provide a summary of Nvidia's most recent quarterly 10-Q filing"),
]
response = client.chat.completions.create(messages=messages, model="jamba-1.5-large", tools=tools)
assistant_message = response.choices[0].message
messages.append(assistant_message)

tool_result = None
tool_calls = assistant_message.tool_calls
if tool_calls:
    tool_call = tool_calls[0]
    if tool_call.function.name == "sec_10q":
        tools_parameters_dict = json.loads(tool_call.function.arguments)
        if "ticker" in tools_parameters_dict:
            tool_result = sec_10q(tools_parameters_dict["ticker"])
        else:
            print("Missing 'ticker' in function arguments")
    else:
        print(f"Unexpected tool call found - {tool_call.function.name}")
else:
    print("No tool calls found")

if tool_result is not None:
    tool_message = ToolMessage(role="tool", tool_call_id=tool_call.id, content=json.dumps(tool_result))
    messages.append(tool_message)
    response_final = client.chat.completions.create(messages=messages, model="jamba-1.5-large", tools=tools)

print("Final response:")
print(response_final.choices[0].message.content)